# CDC: Monkeypox cases, by country

In [1]:
import pandas as pd
import us
import urllib.request, json
import datetime as dt

In [18]:
today = pd.Timestamp.now(tz='America/Los_Angeles').strftime("%Y-%m-%d")
time = pd.Timestamp.now(tz='America/Los_Angeles').strftime("%-I:%M %p")

In [20]:
time

'5:30 AM'

---

## Country populations

#### Figures from World Bank

In [5]:
countries_pop = pd.read_csv('data/raw/countries_population.csv')

---

## CDC Monkeypox

#### Aggregated by country

In [6]:
countries_url = 'https://www.cdc.gov/poxvirus/monkeypox/modules/data-viz/mpx_world_map_prod.json'

In [7]:
with urllib.request.urlopen(countries_url) as url:
    data = json.loads(url.read().decode())
    countries_src = pd.DataFrame(data['data'])

In [8]:
countries_src.columns = countries_src.columns.str.lower()

In [9]:
countries_src.head()

,country,cases,category,asof
0,Argentina,18,Has not historically reported monkeypox,Data as of 22 Jul 2022 5:00 PM EDT
1,Australia,42,Has not historically reported monkeypox,Data as of 22 Jul 2022 5:00 PM EDT
2,Austria,99,Has not historically reported monkeypox,Data as of 22 Jul 2022 5:00 PM EDT
3,Bahamas,1,Has not historically reported monkeypox,Data as of 22 Jul 2022 5:00 PM EDT
4,Barbados,1,Has not historically reported monkeypox,Data as of 22 Jul 2022 5:00 PM EDT


#### Map FIPS codes and AP abbrevations to each state

In [8]:
countries_src['state_fips'] = countries_src['state'].map(us.countries.mapping('name', 'fips'))
countries_src['state_ap'] = countries_src['state'].map(us.countries.mapping('name', 'ap_abbr'))

---

## Merge

#### Population and cases to create a rate

In [9]:
df = pd.merge(countries_src, countries_pop, on=['state_fips', 'state'])

In [10]:
df['cases_per_million'] = ((df['cases'] / df['pop_acs_2020_5tr'])*1000000).astype(float).round(2)

In [11]:
df.drop(['range'], axis=1, inplace=True)

---

## Exports

In [12]:
df.to_csv(f'data/processed/monkeypox_cases_countries_cdc_{today}.csv', index=False)
df.to_json(f'data/processed/monkeypox_cases_countries_cdc_{today}.json', orient='records', indent=4)